In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import importlib

import torch
import torch.nn as nn
from torch_geometric.data import DataLoader

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath('../../code/'))
import utils
from utils.functional_graphs import create_graph_dataset
from utils.models import Attention2Conv
from utils.traning_functions import train_process, test_process

In [ ]:
importlib.reload(utils)

In [ ]:
import warrior as war

In [ ]:
fcs = np.load('../../local/fcs/combined/sparse_fcs_1316_subjects.npy')
print(fcs.shape)
labels = np.load('../../local/gender_labels/combined_gender_labels_1316_subjects.npy')
print(labels.shape)

In [ ]:
N_GRAPHS = fcs.shape[0]
PERCENT = 0.75 
N_TRAINING_SAMPLES = int(np.floor(PERCENT*N_GRAPHS))
#N_TRAINING_SAMPLES, N_TRAINING_SAMPLES/N_GRAPHS

In [ ]:
root = '../../local/graph_datasets/hcp_1316/'
dataset = create_graph_dataset(sparse_fcs=fcs, root=root, labels=labels)

In [ ]:
dataset_shuffled = dataset.shuffle()
train_dataset = dataset_shuffled[:N_TRAINING_SAMPLES]
test_dataset = dataset_shuffled[N_TRAINING_SAMPLES:]
print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

In [ ]:
BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)#, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)#, shuffle=False)

In [ ]:
my_dimensions = [64, 128, 256]
my_learning_rates = [.0005, .0001]
EPOCHS = 101

In [ ]:
for p1 in my_learning_rates:
    for p2 in my_edge_dropout_rate:
        print(f'Paremeters: {p1, p2}')
        model = Attention2Conv(num_node_features=dataset.num_features, hidden_channels=p1, dropout_rate=0.5, edge_dropout_rate=0.1)
        metrics = np.empty((2,EPOCHS))
        optimizer = torch.optim.AdamW(model.parameters(),lr=p2,  weight_decay=1e-4, amsgrad=True)
        criterion = nn.BCEWithLogitsLoss()
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = model.to(device)
        for epoch in range(EPOCHS):
            train_process(model, train_loader, optimizer, criterion, device)
            train_acc = test_process(model, train_loader, device)
            test_acc = test_process(model, test_loader, device)
            metrics[0,epoch] = train_acc
            metrics[1,epoch] = test_acc
            #if epoch % 10 == 0:
                #print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')
        print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')
                
        np.save(f'results/fc_data/metrics/2conv/fc2c_metrics_dim{p1}_lr{p2}.npy', metrics)
        torch.save(model.state_dict(), f'results/fc_data/models/2conv/fc2c_model_dim{p1}_lr{p2}')